In [1]:
# Import python libraries
import numpy as np
import pandas as pd
import welleng as we
import plotly.graph_objects as go
from plotly.offline import plot

C:\Users\gaoqian\AppData\Local\Temp\ipykernel_19672\3359245035.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [21]:
ureg = we.units.ureg

node0 = we.node.Node(pos=[0, 0, 0], md=0, inc=0, azi=0)
node1 = we.node.Node(md=731.5, inc=0, azi=0)
node2 = we.node.Node(md=1280, inc=44, azi=0)
node3 = we.node.Node(md=3718, inc=44, azi=0)

# create an empty list
connectors = []

# append the first connector from the first to the second node
connectors.append(
    we.connector.Connector(node0, node1)
)

# us the newly created node in the connector list and connect it to the third node
connectors.append(
    we.connector.Connector(connectors[-1].node_end, node2, dls_design=2)
)

# similarly connector the current last node in the list to the final node
connectors.append(
    we.connector.Connector(connectors[-1].node_end, node3, dls_design=4)
)

# use the list of connectors as input to create a Survey instance
survey_example_2 = we.survey.from_connections(
    connectors
).interpolate_survey(step=30)

# Create the figure
ureg = we.units.ureg

node0 = we.node.Node(pos=[0, 0, 0], md=0, inc=0, azi=0)
node1 = we.node.Node(md=731.5, inc=0, azi=0)
node2 = we.node.Node(md=1280, inc=44, azi=0)
node3 = we.node.Node(md=3718, inc=44, azi=0)

# create an empty list
connectors = []

# append the first connector from the first to the second node
connectors.append(
    we.connector.Connector(node0, node1)
)

# us the newly created node in the connector list and connect it to the third node
connectors.append(
    we.connector.Connector(connectors[-1].node_end, node2, dls_design=2)
)

# similarly connector the current last node in the list to the final node
connectors.append(
    we.connector.Connector(connectors[-1].node_end, node3, dls_design=4)
)

# use the list of connectors as input to create a Survey instance
survey_example_2 = we.survey.from_connections(
    connectors
).interpolate_survey(step=30)



In [22]:
import plotly.graph_objects as go

# Create a scatter plot for the survey points
fig = go.Figure()

# Add scatter points for the survey nodes
fig.add_trace(go.Scatter3d(
    x=survey_example_2.x,
    y=survey_example_2.y,
    z=survey_example_2.z,
    mode='markers',
    marker=dict(size=2, color='blue'),
    name='Survey Points'
))

# Reverse the z-axis for better visualization
fig.update_scenes(zaxis_autorange="reversed")

# Customize the layout
fig.update_layout(
    scene=dict(
        xaxis_title='East(+)/West(-)',
        yaxis_title='North(+)/South(-)',
        zaxis_title='TVD',
        aspectmode='cube',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
    ),
    width=800,
    height=600,
    margin=dict(l=0, r=0, b=0, t=0)
)

# Show the plot
fig.show()

In [47]:
# initiate a BHA instance
bha = we.architecture.BHA(
    name='8 1/2" Drilling BHA', top=0, bottom=3718, method="bottom_up")

# add some Drill Collars
bha.add_section(
    od=(6.5 * ureg.inches).to('meters').m,
    id=(4 * ureg.inches).to('meters').m,
    length=(372 * ureg.ft).to('meters').m,
    unit_weight=(146.9 * ureg('lbs / ft').to('kg / meters')).m * 9.81,
    name='6 1/2" DC'
)

# add some HWDP
bha.add_section(
    od=(4.5 * ureg.inches).to('meters').m,
    tooljoint_od=(6.375 * ureg.inches).to('meters').m,
    id=(4 * ureg.inches).to('meters').m,
    length=(840 * ureg.ft).to('meters').m,
    unit_weight=(46.90 * ureg('lbs / ft').to('kg / meters')).m * 9.81,
    name='5" HWDP'
)

# add the drill pipe back to surface
bha.add_section(
    od=(5.0 * ureg.inches).to('meters').m,
    tooljoint_od=(6.625 * ureg.inches).to('meters').m,
    id=(3.625 * ureg.inches).to('meters').m,
    length=None,
    unit_weight=(20 * ureg('lbs / ft').to('kg / meters')).m * 9.81,
    name='5" DP'
)

In [43]:
# initiate a WellBore instance
wellbore = we.architecture.WellBore(
    name='8 1/2" Hole Section', top=0, bottom=3718, method='top_down'
)

# add the casing
wellbore.add_section(
    od=9+5/8, id=8.5,
    bottom=3688 * 0.7,
    unit_weight=(68 * ureg('lbs / ft').to('kg / meters')).m,
    coeff_friction_sliding=0.39,
    name='production 9 5/8" casing'
)

# add the open hole section below the casing to the section TD
wellbore.add_section(
    od=None, id=8.5, bottom=3718, unit_weight=None,
    coeff_friction_sliding=0.39, name='8 1/2" OH'
)

In [44]:
hookload = we.torque_drag.HookLoad(
    survey=survey_example_2, wellbore=wellbore, string=bha,
    fluid_density=11.6 / 8.33, step=30,
    name='8 1/2" Hole Section', ff_range=(0.1, 0.4, 0.1)
)


hookload.figure().show()


In [52]:

# Weight on Bit wob, Torque on Bottom/Bit tob and the overpull, all in Newtons

t_and_d = we.torque_drag.TorqueDrag(
    survey=survey_example_2, wellbore=wellbore, string=bha,
    fluid_density=9.8 / 8.33,
    wob=10000, tob=(2000 * 1.356), overpull=50000,
    name='8 1/2" Hole Section'
)


In [55]:
print(dir(t_and_d))


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'add_survey_points_from_strings', 'azi_delta', 'coeff_friction_sliding', 'figure', 'fluid_density', 'get_azi_delta', 'get_buoyancy_factors', 'get_characteristic_od', 'get_coeff_friction_sliding', 'get_forces_and_torsion', 'get_inc_average', 'get_inc_delta', 'get_weight_buoyed_and_radius', 'inc_average', 'inc_delta', 'index', 'name', 'overpull', 'radius', 'string', 'survey', 'survey_original', 'tension', 'tob', 'torque', 'weight_buoyed', 'wellbore', 'wob']


In [69]:
t_and_d.figure().show()

DimensionalityError: Cannot convert from 'number_meter' ([length] / [mass]) to 'ft_lbf' ([length] ** 2 * [mass] / [time] ** 2)